In [1]:
#read a video stream
#extract faces out of it
#load the training data
#use knn to find the face
#map the predicted id to the name of the user
#display predictions

In [7]:
import cv2
import numpy as np
import os

In [8]:
def distance(x,y):
    return np.sqrt(sum((x-y)**2))
def knn(train,test,k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        #Get the vector and the label
        ix=train[i,:-1]
        iy=train[i,-1]
        
        #compute the distance from the test point
        d=distance(test,ix)
        dist.append([d,iy])
    
    dk=sorted(dist,key=lambda x:x[0])[:k]
    #Retrieve only the labels
    labels=np.array(dk)[:,-1]
    #Get the frequencies of each label
    output=np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    pred=output[0][index]
    return pred

In [9]:
cap=cv2.VideoCapture(0)
face=cv2.CascadeClassifier("haarcascade_frontalface.xml")
skip=0
face_data=[]
dataset_path="./data/"
labels=[]
class_id=0 #labels for the given file
name={} #mapping between id-name

# data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        name[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create Labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
        
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
trainset=np.concatenate((face_dataset,face_labels),axis=1)

#testing
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    faces=face.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3],reverse=True)
    
    for(x,y,w,h) in faces:
        #extract the cropped area
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+h+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        #Predict label
        out=knn(trainset,face_section.flatten())
        
        #Display on screen
        pred_name=name[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        
    cv2.imshow('Faces',frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
